# Training and test set predictions for TFIDFVectorizer

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from typing import List, Dict, Optional, Tuple, Any
import numpy as np
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, BatchNormalization
import tensorflow as tf
from sklearn.metrics import accuracy_score, f1_score
from sklearn.metrics import classification_report
from sklearn.model_selection import KFold

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/')

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


In [ ]:
%cd /content/gdrive/MyDrive/Masters/Year_1/Semester_2/CS_505/Project/

/content/gdrive/MyDrive/Masters/Year_1/Semester_2/CS_505/Project


In [ ]:
train_df = pd.read_csv('Data/Train.csv')

In [ ]:
le = LabelEncoder()
train_df.Label = le.fit_transform(train_df.Label)

In [ ]:
le.classes_

array(['ARTS AND CRAFTS', 'CULTURE', 'ECONOMY', 'EDUCATION', 'FARMING',
       'FLOODING', 'HEALTH', 'LAW/ORDER', 'LOCALCHIEFS', 'MUSIC',
       'OPINION/ESSAY', 'POLITICS', 'RELATIONSHIPS', 'RELIGION', 'SOCIAL',
       'SOCIAL ISSUES', 'SPORTS', 'TRANSPORT', 'WILDLIFE/ENVIRONMENT',
       'WITCHCRAFT'], dtype=object)

In [ ]:
Y_train = np_utils.to_categorical(train_df['Label'], np.unique(train_df['Label']).shape[0])

In [ ]:
def get_model(size):
  print('Building model...')
  model = Sequential()
  model.add(Dense(2048, input_shape=(size,)))
  model.add(Activation('relu'))
  model.add(Dropout(0.2))
  model.add(Dense(20))
  model.add(Activation('softmax'))

  model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
  return model
  #make one, takes in, double that amount, english and chichewa

In [ ]:
def get_results(model,epochs,train_dataset,test_dataset,y_test):
  model.fit(train_dataset,
            epochs=epochs, batch_size=32,
            verbose=1)
  print(model.evaluate(test_dataset))
  ans = model.predict(test_dataset)
  print(accuracy_score(y_test, np_utils.to_categorical(np.argmax(ans,axis=1),num_classes=20)))
  print(classification_report(y_test, np_utils.to_categorical(np.argmax(ans,axis=1),num_classes=20)))

In [ ]:
print(Y_train.shape)
print(train_df.shape)

(1436, 20)
(1436, 3)


In [ ]:
test_df = pd.read_csv('Data/Test.csv')

In [ ]:
test_df.head()

,ID,Text
0,ID_ADHEtjTi,Abambo odzikhweza akuchuluka Kafukufuku wa ap...
1,ID_AHfJktdQ,Ambuye Ziyaye Ayamikira Aphunzitsi a Tilitonse...
2,ID_AUJIHpZr,Anatcheleza: Akundiopseza a gogo wanga Akundi...
3,ID_AUKYBbIM,Ulova wafika posauzana Adatenga digiri ya uph...
4,ID_AZnsVPEi,"Dzombe kukoma, koma Kuyambira makedzana, pant..."


In [ ]:
Y_train = np_utils.to_categorical(train_df['Label'], np.unique(train_df['Label']).shape[0])

In [ ]:
print(train_df.shape)
print(Y_train.shape)

(1436, 3)
(1436, 20)


In [ ]:
tfidf = TfidfVectorizer()
tfidf_train = tfidf.fit_transform(train_df.Text).toarray()
#tfidf_test = tfidf.transform(X_test).toarray()
train_dataset = tf.data.Dataset.from_tensor_slices((tfidf_train, Y_train))

#test_dataset = tf.data.Dataset.from_tensor_slices((tfidf_test, y_test))

BATCH_SIZE = 32
SHUFFLE_BUFFER_SIZE = 100

train_dataset = train_dataset.shuffle(SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE)
#test_dataset = test_dataset.batch(BATCH_SIZE)
model_ = get_model(tfidf_train.shape[1])

model_.fit(train_dataset,epochs=50, batch_size=32,verbose=1)

Building model...
Epoch 1/50
45/45 [==============================] - 4s 41ms/step - loss: 2.3116 - accuracy: 0.3175
Epoch 2/50
45/45 [==============================] - 2s 39ms/step - loss: 0.7793 - accuracy: 0.8565
Epoch 3/50
45/45 [==============================] - 2s 40ms/step - loss: 0.1235 - accuracy: 0.9916
Epoch 4/50
45/45 [==============================] - 2s 34ms/step - loss: 0.0269 - accuracy: 0.9965
Epoch 5/50
45/45 [==============================] - 1s 32ms/step - loss: 0.0148 - accuracy: 0.9965
Epoch 6/50
45/45 [==============================] - 1s 32ms/step - loss: 0.0120 - accuracy: 0.9965
Epoch 7/50
45/45 [==============================] - 1s 32ms/step - loss: 0.0094 - accuracy: 0.9972
Epoch 8/50
45/45 [==============================] - 1s 32ms/step - loss: 0.0089 - accuracy: 0.9972
Epoch 9/50
45/45 [==============================] - 1s 32ms/step - loss: 0.0078 - accuracy: 0.9972
Epoch 10/50
45/45 [==============================] - 1s 32ms/step - loss: 0.0078 - accuracy

In [ ]:
tfidf_test = tfidf.transform(test_df.Text).toarray()
test_dataset = tf.data.Dataset.from_tensor_slices((tfidf_test))

In [ ]:
print(test_df.shape)

(620, 2)


In [ ]:
ans = model_.predict(tfidf_test)

In [ ]:
print(ans.shape)
ans_arr = np.argmax(ans,axis=1)

(620, 20)


In [ ]:
ans_arr[:2]

array([15, 13])

In [ ]:
le.classes_

array(['ARTS AND CRAFTS', 'CULTURE', 'ECONOMY', 'EDUCATION', 'FARMING',
       'FLOODING', 'HEALTH', 'LAW/ORDER', 'LOCALCHIEFS', 'MUSIC',
       'OPINION/ESSAY', 'POLITICS', 'RELATIONSHIPS', 'RELIGION', 'SOCIAL',
       'SOCIAL ISSUES', 'SPORTS', 'TRANSPORT', 'WILDLIFE/ENVIRONMENT',
       'WITCHCRAFT'], dtype=object)

In [ ]:
arr_class = list(le.inverse_transform(ans_arr))

In [ ]:
arr_class[:5]

['SOCIAL ISSUES', 'RELIGION', 'RELATIONSHIPS', 'POLITICS', 'SOCIAL ISSUES']

In [ ]:
test_df_copy = test_df.copy()

In [ ]:
print(test_df_copy)

              ID                                               Text
0    ID_ADHEtjTi   Abambo odzikhweza akuchuluka Kafukufuku wa ap...
1    ID_AHfJktdQ  Ambuye Ziyaye Ayamikira Aphunzitsi a Tilitonse...
2    ID_AUJIHpZr   Anatcheleza: Akundiopseza a gogo wanga Akundi...
3    ID_AUKYBbIM   Ulova wafika posauzana Adatenga digiri ya uph...
4    ID_AZnsVPEi   Dzombe kukoma, koma Kuyambira makedzana, pant...
..           ...                                                ...
615  ID_zdpOUWyJ  Kanyongolo Wapempha Oyimira Milandu Kuti Atsat...
616  ID_zhnOomuu  Amandimenya\nZikomo gogo,\nNdine mtsikana wa z...
617  ID_zmWHvBJb   Apolisi athotha gulu la MYP Asilikali 56 a gu...
618  ID_zphjdFIb   Mwambo wa ukwati wa Chitonga Mtundu wina uliw...
619  ID_ztdtrNxt   Mwapasa autsa mapiri Pamene pali kusamvana pa...

[620 rows x 2 columns]


In [ ]:
test_df_copy["Labels"] = arr_class

In [ ]:
print(test_df_copy)

              ID                                               Text  \
0    ID_ADHEtjTi   Abambo odzikhweza akuchuluka Kafukufuku wa ap...   
1    ID_AHfJktdQ  Ambuye Ziyaye Ayamikira Aphunzitsi a Tilitonse...   
2    ID_AUJIHpZr   Anatcheleza: Akundiopseza a gogo wanga Akundi...   
3    ID_AUKYBbIM   Ulova wafika posauzana Adatenga digiri ya uph...   
4    ID_AZnsVPEi   Dzombe kukoma, koma Kuyambira makedzana, pant...   
..           ...                                                ...   
615  ID_zdpOUWyJ  Kanyongolo Wapempha Oyimira Milandu Kuti Atsat...   
616  ID_zhnOomuu  Amandimenya\nZikomo gogo,\nNdine mtsikana wa z...   
617  ID_zmWHvBJb   Apolisi athotha gulu la MYP Asilikali 56 a gu...   
618  ID_zphjdFIb   Mwambo wa ukwati wa Chitonga Mtundu wina uliw...   
619  ID_ztdtrNxt   Mwapasa autsa mapiri Pamene pali kusamvana pa...   

            Labels  
0    SOCIAL ISSUES  
1         RELIGION  
2    RELATIONSHIPS  
3         POLITICS  
4    SOCIAL ISSUES  
..             ...  


In [ ]:
subm = pd.read_csv('Data/SampleSubmission.csv')

In [ ]:
subm.head()

,ID,Label
0,ID_sQaPRMWO,0
1,ID_TanclvfR,0
2,ID_CNbveyvk,0
3,ID_MclKMhyP,0
4,ID_rNrmXOGD,0


In [ ]:
subm.drop(columns=['Label'],inplace=True)

In [ ]:
n_df = pd.merge(subm,test_df_copy, left_on='ID', right_on='ID')

In [ ]:
n_df.rename(columns={"Labels": "Label"},inplace=True)

In [ ]:
n_df.head()

,ID,Text,Label
0,ID_sQaPRMWO,Two Sentenced to 5 Years for Armed Robbery Man...,LAW/ORDER
1,ID_TanclvfR,Papa Wakhazikitsa Bungwe Lolimbikitsa Umodzi W...,SOCIAL
2,ID_CNbveyvk,Amalawi a nsomba ali kakasi ku Botswana Ali m...,POLITICS
3,ID_MclKMhyP,Mtumiki wa Mulungu adalosera Onse okhulupirir...,SOCIAL ISSUES
4,ID_rNrmXOGD,Kubwera kwa 1 000 Kwacha kukusonyeza sizikuye...,ECONOMY


In [ ]:
n_df.to_csv('Data/MLP_Submission.csv',columns=['ID','Label'],index=False)